In [1]:
# トップレートパーティの相互対戦のバトルログ可視化

In [2]:
from bson import ObjectId
import numpy as np
from pokeai.util import json_load, pickle_load, ROOT_DIR, DATASET_DIR, pickle_base64_loads
import json
import re

In [3]:
name2jp = json_load(DATASET_DIR.joinpath("name2jp.json"))["name2jp"]

In [47]:
re_name2jp = re.compile("\\b("+"|".join(name2jp.keys())+")\\b")

In [48]:
def text2jp(text):
    return re_name2jp.sub(lambda m: name2jp[m.group()],text)

In [8]:
with open(r"D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815\top_player_rating_battle.log") as f:
    lines = f.readlines()

In [9]:
print(list(enumerate(lines[:100])))

[(0, 'DEBUG:__main__:match start: 5f37a6829949403bf6af940e+5f3835dad10c9203103a6829, 5f37a6829949403bf6af9413+5f3903f5dcc450c616728c8f\n'), (1, 'DEBUG:pokeai.sim.sim:writeChunk ">start {\\"formatid\\": \\"gen2customgame\\"}\\n>player p1 {\\"name\\": \\"p1\\", \\"team\\": \\"snorlax|||noability|surf,skullbash,rocksmash,bodyslam||255,255,255,255,255,255|M|30,30,30,30,30,30||55|\\"}\\n>player p2 {\\"name\\": \\"p2\\", \\"team\\": \\"zapdos|||noability|thunder,steelwing,drillpeck,skyattack||255,255,255,255,255,255|N|30,30,30,30,30,30||55|\\"}"\n'), (2, 'DEBUG:pokeai.sim.sim:readChunk "sideupdate\\np1\\n|request|{\\"active\\":[{\\"moves\\":[{\\"move\\":\\"Surf\\",\\"id\\":\\"surf\\",\\"pp\\":24,\\"maxpp\\":24,\\"target\\":\\"allAdjacentFoes\\",\\"disabled\\":false},{\\"move\\":\\"Skull Bash\\",\\"id\\":\\"skullbash\\",\\"pp\\":24,\\"maxpp\\":24,\\"target\\":\\"normal\\",\\"disabled\\":false},{\\"move\\":\\"Rock Smash\\",\\"id\\":\\"rocksmash\\",\\"pp\\":24,\\"maxpp\\":24,\\"target\\":\\"nor

In [10]:
# match startの行でどのパーティが対戦しているかを取得、またパーティ内容があるので表示
# updateの行で起こったことがかかれているので日本語化して表示
# choice_turn_startでプレイヤーの状態と選択肢を表示
# match endの行で勝者を表示して終了

In [57]:
def extract_json(line):
    return line[line.find("{"):line.rfind("}")+1]

In [66]:
def print_status(status, q_func_data):
    battle_status = pickle_base64_loads(status["battle_status"])
    def rank_str(st):
        s = ""
        for k, v in st.ranks.items():
            if v != 0:
                s += f" {k}{v:+}"
        return s
    st = battle_status.side_statuses[battle_status.side_friend]
    friend_side_txt = f'自分 {name2jp[st.active.species]} HP{st.active.hp_current}/{st.active.hp_max} {st.active.status} {rank_str(st.active)}'
    st = battle_status.side_statuses[battle_status.side_opponent]
    opponent_side_txt = f'相手 {name2jp[st.active.species]} HP{st.active.hp_current}/{st.active.hp_max} {st.active.status} {rank_str(st.active)}'
    print(opponent_side_txt)
    print(friend_side_txt)
    #if q_func_data is None:
    #    print("no Q")
    #    return
    #print("技　　　　　　　Q値")
    #for m,q in zip(battle_status.side_party[0]["moves"], q_func_data):
    #    print(name2jp[m].ljust(8, "　")+f"{q:.02f}")
    print([name2jp[m] for m in battle_status.side_party[0]["moves"]])
    if len(status["request"]["active"][0]["moves"])==1:
        print("選択不可状態")

In [59]:
def print_status_choice(status, q_func_data):
    print_status(status, q_func_data)
    choice = status["choice"]#move [1-4]
    move_idx = int(choice[5])-1
    battle_status = pickle_base64_loads(status["battle_status"])
    print(f'選択=> {name2jp[battle_status.side_party[0]["moves"][move_idx]]}')

In [60]:
def get_q_func(line):
    data = json.loads(extract_json(line))
    return data["q_func"][:4]

In [17]:
match_start_prefix = "DEBUG:__main__:match start: "
status_log_prefix = "DEBUG:pokeai.sim.battle_stream_processor:choice_turn_start:"
qfunc_log_prefix = "DEBUG:pokeai.ai.generic_move_model.agent:q_func:"
update_head = "DEBUG:pokeai.sim.sim:readChunk \"update"
update_prefix = "DEBUG:pokeai.sim.sim:readChunk "
match_end_prefix = "DEBUG:__main__:match end:"

In [55]:
def display_target_battles(target_pattern):
    target_match_re = re.compile(target_pattern)
    in_target_battle = False
    battle_count = 0
    last_q_data = None
    for line in lines:
        if line.startswith(match_start_prefix):
            if target_match_re.search(line):
                in_target_battle = True
                battle_count += 1
                if battle_count > 100:
                    break
                print(line)
            else:
                in_target_battle = False
        if in_target_battle:
            #if line.startswith(qfunc_log_prefix):
            #    last_q_data = get_q_func(line)
            if line.startswith(update_head):
                print(text2jp(json.loads(line[len(update_prefix):])))
            if line.startswith(status_log_prefix):
                status = json.loads(extract_json(line))
                print_status_choice(status, last_q_data)
                last_q_data = None
                print()

In [67]:
display_target_battles("5f37a6829949403bf6af9412\\+5f38dbeb63569b3772554eba")

DEBUG:__main__:match start: 5f37a6829949403bf6af9412+5f38dbeb63569b3772554eba, 5f37a6829949403bf6af940f+5f385e2797c1e7ac0c795b7e

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ドンファン|ドンファン, L55, M|215/215
|switch|p1a: ドンファン|ドンファン, L55, M|215/215
|split|p2
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|turn|1
相手 ケンタロス HP198/198  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP215/215  
自分 ケンタロス HP198/198  
['つのドリル', 'ずつき', 'かえんほうしゃ', 'じしん']
選択=> じしん

update
|
|move|p2a: ケンタロス|じしん|p1a: ドンファン
|split|p1
|-damage|p1a: ドンファン|178/215
|-damage|p1a: ドンファン|178/215
|move|p1a: ドンファン|じしん|p2a: ケンタロス
|-crit|p2a: ケンタロス
|split|p2
|-damage|p2a: ケンタロス|48/198
|-damage|p2a: ケンタロス|48/198
|
|upkeep
|turn|2
相手 ケンタロス HP48/198  
自分 ドンファン HP178/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じ


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ドンファン|ドンファン, L55, M|215/215
|switch|p1a: ドンファン|ドンファン, L55, M|215/215
|split|p2
|switch|p2a: ゴルダック|ゴルダック, L55, M|204/204
|switch|p2a: ゴルダック|ゴルダック, L55, M|204/204
|turn|1
相手 ゴルダック HP204/204  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP215/215  
自分 ゴルダック HP204/204  
['どろかけ', 'のしかかり', 'かげぶんしん', 'なみのり']
選択=> なみのり

update
|
|move|p2a: ゴルダック|なみのり|p1a: ドンファン
|-supereffective|p1a: ドンファン
|split|p1
|-damage|p1a: ドンファン|55/215
|-damage|p1a: ドンファン|55/215
|move|p1a: ドンファン|じしん|p2a: ゴルダック
|split|p2
|-damage|p2a: ゴルダック|114/204
|-damage|p2a: ゴルダック|114/204
|
|upkeep
|turn|2
相手 ゴルダック HP114/204  
自分 ドンファン HP55/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP55/215  
自分 ゴルダック HP114/204  
['どろかけ', 'のしかかり', 'かげぶんしん', 'なみのり']
選択=> なみのり

update
|
|move|p2a: ゴルダッ

|win|p2
DEBUG:__main__:match start: 5f37a6829949403bf6af940f+5f385e2797c1e7ac0c795ba4, 5f37a6829949403bf6af9412+5f38dbeb63569b3772554eba

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ライコウ|ライコウ, L55|215/215
|switch|p1a: ライコウ|ライコウ, L55|215/215
|split|p2
|switch|p2a: ドンファン|ドンファン, L55, M|215/215
|switch|p2a: ドンファン|ドンファン, L55, M|215/215
|turn|1
相手 ドンファン HP215/215  
自分 ライコウ HP215/215  
['いわくだき', 'かいりき', 'いあいぎり', '１０まんボルト']
選択=> １０まんボルト

相手 ライコウ HP215/215  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

update
|
|move|p1a: ライコウ|１０まんボルト|p2a: ドンファン
|-immune|p2a: ドンファン
|move|p2a: ドンファン|じしん|p1a: ライコウ
|-supereffective|p1a: ライコウ
|split|p1
|-damage|p1a: ライコウ|33/215
|-damage|p1a: ライコウ|33/215
|
|upkeep
|turn|2
相手 ドンファン HP215/215  
自分 ライコウ HP33/215  
['いわくだき', 'かいりき', 'いあいぎり', '１０まんボルト']
選択=> かいりき

相手 ライコウ HP33/215  
自分 ドンファン 

|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af9410+5f3888187955353d68b9e656, 5f37a6829949403bf6af9412+5f38dbeb63569b3772554eba

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カイリュー|カイリュー, L55, M|216/216
|switch|p1a: カイリュー|カイリュー, L55, M|216/216
|split|p2
|switch|p2a: ドンファン|ドンファン, L55, M|215/215
|switch|p2a: ドンファン|ドンファン, L55, M|215/215
|turn|1
相手 ドンファン HP215/215  
自分 カイリュー HP216/216  
['れいとうパンチ', 'つのドリル', '１０まんボルト', 'つばさでうつ']
選択=> れいとうパンチ

相手 カイリュー HP216/216  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> げんしのちから

update
|
|move|p1a: カイリュー|れいとうパンチ|p2a: ドンファン
|-supereffective|p2a: ドンファン
|split|p2
|-damage|p2a: ドンファン|121/215
|-damage|p2a: ドンファン|121/215
|move|p2a: ドンファン|げんしのちから|p1a: カイリュー
|-supereffective|p1a: カイリュー
|split|p1
|-damage|p1a: カイリュー|151/216
|-damage|p1a: カイリュー|151/216
|
|upkeep
|turn|2
相手 ドンファン HP1

|turn|1
相手 ドンファン HP215/215  
自分 ライコウ HP215/215  
['かげぶんしん', 'スピードスター', '１０まんボルト', 'いあいぎり']
選択=> スピードスター

相手 ライコウ HP215/215  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

update
|
|move|p1a: ライコウ|スピードスター|p2a: ドンファン
|split|p2
|-damage|p2a: ドンファン|193/215
|-damage|p2a: ドンファン|193/215
|move|p2a: ドンファン|じしん|p1a: ライコウ
|-supereffective|p1a: ライコウ
|split|p1
|-damage|p1a: ライコウ|41/215
|-damage|p1a: ライコウ|41/215
|
|upkeep
|turn|2
相手 ドンファン HP193/215  
自分 ライコウ HP41/215  
['かげぶんしん', 'スピードスター', '１０まんボルト', 'いあいぎり']
選択=> １０まんボルト

相手 ライコウ HP41/215  
自分 ドンファン HP193/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

update
|
|move|p1a: ライコウ|１０まんボルト|p2a: ドンファン
|-immune|p2a: ドンファン
|move|p2a: ドンファン|じしん|p1a: ライコウ
|-supereffective|p1a: ライコウ
|split|p1
|-damage|p1a: ライコウ|0 fnt
|-damage|p1a: ライコウ|0 fnt
|faint|p1a: ライコウ
|
|win|p2
DEBUG:__main__:match start: 5f37a6829949403bf6af9414+5f392feeba887aa6e5443f2d, 5f37a6829949403bf6af9412+5f38dbeb63569b3772554eba

update
|player|p1|p1||
|player|p2|p2||


|turn|2
相手 ラプラス HP221/259  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP215/215  
自分 ラプラス HP221/259  
['バブルこうせん', 'サイコキネシス', 'つのドリル', 'ふぶき']
選択=> サイコキネシス

update
|
|move|p2a: ラプラス|サイコキネシス|p1a: ドンファン
|split|p1
|-damage|p1a: ドンファン|162/215
|-damage|p1a: ドンファン|162/215
|move|p1a: ドンファン|じしん|p2a: ラプラス
|split|p2
|-damage|p2a: ラプラス|136/259
|-damage|p2a: ラプラス|136/259
|
|upkeep
|turn|3
相手 ラプラス HP136/259  
自分 ドンファン HP162/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP162/215  
自分 ラプラス HP136/259  
['バブルこうせん', 'サイコキネシス', 'つのドリル', 'ふぶき']
選択=> サイコキネシス

update
|
|move|p2a: ラプラス|サイコキネシス|p1a: ドンファン
|split|p1
|-damage|p1a: ドンファン|114/215
|-damage|p1a: ドンファン|114/215
|move|p1a: ドンファン|じしん|p2a: ラプラス
|split|p2
|-damage|p2a: ラプラス|43/259
|-damage|p2a: ラプラス|43/259
|
|upkeep
|turn|4
相手 ラプラス HP43/259  
自分 ドンファン HP114/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP114/215  
自分 ラプラス HP43/259  
['バブルこうせん', 'サイコキネシス', 'つのドリル', 'ふぶき']
選択=> サイコキネシス

update


|turn|3
相手 ドンファン HP91/215  
自分 カビゴン HP81/292  
['じしん', 'すなあらし', 'のしかかり', 'れいとうパンチ']
選択=> じしん

相手 カビゴン HP81/292  
自分 ドンファン HP91/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

update
|
|move|p2a: ドンファン|じしん|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|0 fnt
|-damage|p1a: カビゴン|0 fnt
|faint|p1a: カビゴン
|
|win|p2
DEBUG:__main__:match start: 5f37a6829949403bf6af9412+5f38dbeb63569b3772554eba, 5f37a6829949403bf6af940f+5f385e2797c1e7ac0c7959c0

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ドンファン|ドンファン, L55, M|215/215
|switch|p1a: ドンファン|ドンファン, L55, M|215/215
|split|p2
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|turn|1
相手 ケンタロス HP198/198  
自分 ドンファン HP215/215  
['じしん', 'げんしのちから', 'ころがる', 'つのでつく']
選択=> じしん

相手 ドンファン HP215/215  
自分 ケンタロス HP198/198  
['すてみタックル', 'だいもんじ', 'つのドリル', 'いわくだき']
選択=> すてみタックル

update
|

In [68]:
display_target_battles("5f37a6829949403bf6af9414\\+5f392feeba887aa6e5443e28")

DEBUG:__main__:match start: 5f37a6829949403bf6af940e+5f3835dad10c9203103a66d7, 5f37a6829949403bf6af9414+5f392feeba887aa6e5443e28

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p1a: ケンタロス|ケンタロス, L55, M|198/198
|split|p2
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|turn|1
相手 ケンタロス HP198/198  
自分 ケンタロス HP198/198  
['だいもんじ', 'つのドリル', 'じしん', 'のしかかり']
選択=> つのドリル

相手 ケンタロス HP198/198  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

update
|
|move|p2a: ケンタロス|つのドリル|p1a: ケンタロス|[miss]
|-miss|p2a: ケンタロス
|move|p1a: ケンタロス|つのドリル|p2a: ケンタロス
|split|p2
|-damage|p2a: ケンタロス|0 fnt
|-damage|p2a: ケンタロス|0 fnt
|-ohko
|faint|p2a: ケンタロス
|
|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af940c+5f37d87e4e3f9365cecaca68, 5f37a6829949403bf6af9414+5f392feeba887aa


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p1a: ケンタロス|ケンタロス, L55, M|198/198
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP198/198  
自分 カビゴン HP292/292  
['でんじほう', 'バブルこうせん', 'ちきゅうなげ', 'すてみタックル']
選択=> すてみタックル

update
|
|move|p1a: ケンタロス|つのドリル|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|0 fnt
|-damage|p2a: カビゴン|0 fnt
|-ohko
|faint|p2a: カビゴン
|
|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af9414+5f392feeba887aa6e5443e28, 5f37a6829949403bf6af940e+5f3835dad10c9203103a68c2

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|sta

|turn|1
相手 カビゴン HP292/292  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP198/198  
自分 カビゴン HP292/292  
['ほのおのパンチ', 'のしかかり', 'でんじほう', 'ふぶき']
選択=> のしかかり

update
|
|move|p1a: ケンタロス|つのドリル|p2a: カビゴン|[miss]
|-miss|p1a: ケンタロス
|move|p2a: カビゴン|のしかかり|p1a: ケンタロス
|split|p1
|-damage|p1a: ケンタロス|133/198
|-damage|p1a: ケンタロス|133/198
|-status|p1a: ケンタロス|par
|
|upkeep
|turn|2
相手 カビゴン HP292/292  
自分 ケンタロス HP133/198 par 
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP133/198 par 
自分 カビゴン HP292/292  
['ほのおのパンチ', 'のしかかり', 'でんじほう', 'ふぶき']
選択=> のしかかり

update
|
|move|p2a: カビゴン|のしかかり|p1a: ケンタロス
|split|p1
|-damage|p1a: ケンタロス|66/198 par
|-damage|p1a: ケンタロス|66/198 par
|move|p1a: ケンタロス|つのドリル|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|0 fnt
|-damage|p2a: カビゴン|0 fnt
|-ohko
|faint|p2a: カビゴン
|
|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af9414+5f392feeba887aa6e5443e28, 5f37a6829949403bf6af940e+5f3835dad10c9203103a678f

update
|player|p1|p1||
|player|p2|p2||
|teamsize|

|turn|1
相手 ケンタロス HP198/198  
自分 ラプラス HP259/259  
['ハイドロポンプ', 'ソーラービーム', 'のしかかり', 'つのドリル']
選択=> つのドリル

相手 ラプラス HP259/259  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

update
|
|move|p2a: ケンタロス|つのドリル|p1a: ラプラス
|split|p1
|-damage|p1a: ラプラス|0 fnt
|-damage|p1a: ラプラス|0 fnt
|-ohko
|faint|p1a: ラプラス
|
|win|p2
DEBUG:__main__:match start: 5f37a6829949403bf6af9412+5f38dbeb63569b37725550d8, 5f37a6829949403bf6af9414+5f392feeba887aa6e5443e28

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|split|p2
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p2a: ケンタロス|ケンタロス, L55, M|198/198
|turn|1
相手 ケンタロス HP198/198  
自分 カビゴン HP292/292  
['ふぶき', 'とっしん', 'ころがる', 'のしかかり']
選択=> のしかかり

相手 カビゴン HP292/292  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル


|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af9414+5f392feeba887aa6e5443e28, 5f37a6829949403bf6af9410+5f3888187955353d68b9e4f8

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ケンタロス|ケンタロス, L55, M|198/198
|switch|p1a: ケンタロス|ケンタロス, L55, M|198/198
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP198/198  
自分 カビゴン HP292/292  
['れいとうパンチ', 'バブルこうせん', 'のしかかり', 'いわくだき']
選択=> のしかかり

update
|
|move|p1a: ケンタロス|つのドリル|p2a: カビゴン|[miss]
|-miss|p1a: ケンタロス
|move|p2a: カビゴン|のしかかり|p1a: ケンタロス
|split|p1
|-damage|p1a: ケンタロス|138/198
|-damage|p1a: ケンタロス|138/198
|
|upkeep
|turn|2
相手 カビゴン HP292/292  
自分 ケンタロス HP138/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP138/198  
自分 カビゴン HP292/29

|turn|1
相手 ライコウ HP215/215  
自分 ケンタロス HP198/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP198/198  
自分 ライコウ HP215/215  
['１０まんボルト', 'ずつき', 'どろかけ', 'すなあらし']
選択=> １０まんボルト

update
|
|move|p2a: ライコウ|１０まんボルト|p1a: ケンタロス
|split|p1
|-damage|p1a: ケンタロス|106/198
|-damage|p1a: ケンタロス|106/198
|move|p1a: ケンタロス|つのドリル|p2a: ライコウ|[miss]
|-miss|p1a: ケンタロス
|
|upkeep
|turn|2
相手 ライコウ HP215/215  
自分 ケンタロス HP106/198  
['つのドリル', 'のしかかり', 'どくどく', 'つのでつく']
選択=> つのドリル

相手 ケンタロス HP106/198  
自分 ライコウ HP215/215  
['１０まんボルト', 'ずつき', 'どろかけ', 'すなあらし']
選択=> １０まんボルト

update
|
|move|p2a: ライコウ|１０まんボルト|p1a: ケンタロス
|split|p1
|-damage|p1a: ケンタロス|11/198
|-damage|p1a: ケンタロス|11/198
|move|p1a: ケンタロス|つのドリル|p2a: ライコウ
|split|p2
|-damage|p2a: ライコウ|0 fnt
|-damage|p2a: ライコウ|0 fnt
|-ohko
|faint|p2a: ライコウ
|
|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af9414+5f392feeba887aa6e5443e28, 5f37a6829949403bf6af940f+5f385e2797c1e7ac0c795a00

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|si

In [69]:
display_target_battles("5f37a6829949403bf6af940e\\+5f3835dad10c9203103a67e8")

DEBUG:__main__:match start: 5f37a6829949403bf6af940d+5f3807d4df2e52bb36a250b9, 5f37a6829949403bf6af940e+5f3835dad10c9203103a67e8

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|split|p2
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|turn|1
相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['かえんほうしゃ', 'ころがる', 'すてみタックル', 'いわくだき']
選択=> すてみタックル

相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

update
|
|move|p2a: バンギラス|ばくれつパンチ|p1a: カビゴン|[miss]
|-miss|p2a: バンギラス
|move|p1a: カビゴン|すてみタックル|p2a: バンギラス
|-resisted|p2a: バンギラス
|split|p2
|-damage|p2a: バンギラス|184/226
|-damage|p2a: バンギラス|184/226
|split|p1
|-damage|p1a: カビゴン|282/292|[from] Recoil|[of] p2a: バンギラス
|-damage|p1a: カビゴン|282/292|[from] Recoil|[of] p

|turn|2
相手 カビゴン HP292/292  
自分 バンギラス HP207/226   accuracy-1
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

相手 バンギラス HP207/226   accuracy-1
自分 カビゴン HP292/292  
['どろかけ', 'ころがる', 'ロケットずつき', 'のしかかり']
選択=> どろかけ

update
|
|move|p1a: バンギラス|ばくれつパンチ|p2a: カビゴン
|-supereffective|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|136/292
|-damage|p2a: カビゴン|136/292
|-start|p2a: カビゴン|ねんりき
|-activate|p2a: カビゴン|ねんりき
|move|p2a: カビゴン|どろかけ|p1a: バンギラス
|-supereffective|p1a: バンギラス
|split|p1
|-damage|p1a: バンギラス|187/226
|-damage|p1a: バンギラス|187/226
|-unboost|p1a: バンギラス|accuracy|1
|
|upkeep
|turn|3
相手 カビゴン HP136/292  
自分 バンギラス HP187/226   accuracy-2
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> げんしのちから

相手 バンギラス HP187/226   accuracy-2
自分 カビゴン HP136/292  
['どろかけ', 'ころがる', 'ロケットずつき', 'のしかかり']
選択=> どろかけ

update
|
|move|p1a: バンギラス|げんしのちから|p2a: カビゴン|[miss]
|-miss|p1a: バンギラス
|-activate|p2a: カビゴン|ねんりき
|split|p2
|-damage|p2a: カビゴン|108/292|[from] ねんりき
|-damage|p2a: カビゴン|108/292|[from] ねんりき
|
|upkeep
|turn|4
相手 カビゴン H


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['なみのり', 'かえんほうしゃ', 'いわくだき', 'のしかかり']
選択=> なみのり

update
|
|move|p1a: バンギラス|ばくれつパンチ|p2a: カビゴン|[miss]
|-miss|p1a: バンギラス
|move|p2a: カビゴン|なみのり|p1a: バンギラス
|-supereffective|p1a: バンギラス
|split|p1
|-damage|p1a: バンギラス|161/226
|-damage|p1a: バンギラス|161/226
|
|upkeep
|turn|2
相手 カビゴン HP292/292  
自分 バンギラス HP161/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

相手 バンギラス HP161/226  
自分 カビゴン HP292/292  
['なみのり', 'かえんほうしゃ', 'いわくだき', 'のしかかり']
選択=> なみのり

update
|
|move|p1a: バンギラス|ばくれつパンチ|p2a

|turn|2
相手 カビゴン HP139/292  
自分 バンギラス HP189/226 par 
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> げんしのちから

相手 バンギラス HP189/226 par 
自分 カビゴン HP139/292  
['おんがえし', 'かみなりパンチ', 'でんじほう', 'ふぶき']
選択=> でんじほう

update
|
|-end|p2a: カビゴン|ねんりき
|move|p2a: カビゴン|でんじほう|p1a: バンギラス|[miss]
|-miss|p2a: カビゴン
|move|p1a: バンギラス|げんしのちから|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|71/292
|-damage|p2a: カビゴン|71/292
|
|upkeep
|turn|3
相手 カビゴン HP71/292  
自分 バンギラス HP189/226 par 
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> かえんほうしゃ

相手 バンギラス HP189/226 par 
自分 カビゴン HP71/292  
['おんがえし', 'かみなりパンチ', 'でんじほう', 'ふぶき']
選択=> ふぶき

update
|
|move|p2a: カビゴン|ふぶき|p1a: バンギラス
|split|p1
|-damage|p1a: バンギラス|144/226 par
|-damage|p1a: バンギラス|144/226 par
|cant|p1a: バンギラス|par
|
|upkeep
|turn|4
相手 カビゴン HP71/292  
自分 バンギラス HP144/226 par 
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> げんしのちから

相手 バンギラス HP144/226 par 
自分 カビゴン HP71/292  
['おんがえし', 'かみなりパンチ', 'でんじほう', 'ふぶき']
選択=> でんじほう

update
|
|move|p2a: カビゴン|でんじほう|p1a: バンギラス|[miss]
|-miss

|turn|5
相手 カビゴン HP26/292  
自分 バンギラス HP145/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> １０まんボルト

相手 バンギラス HP145/226  
自分 カビゴン HP26/292  
['のしかかり', 'ころがる', 'ソーラービーム', 'はかいこうせん']
選択=> のしかかり

update
|
|move|p1a: バンギラス|１０まんボルト|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|0 fnt
|-damage|p2a: カビゴン|0 fnt
|faint|p2a: カビゴン
|
|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af940e+5f3835dad10c9203103a67e8, 5f37a6829949403bf6af940f+5f385e2797c1e7ac0c795cca

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['かみなり', 'おんがえし', 'かげぶんしん', 'ソ


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カイリュー|カイリュー, L55, M|216/216
|switch|p1a: カイリュー|カイリュー, L55, M|216/216
|split|p2
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|turn|1
相手 バンギラス HP226/226  
自分 カイリュー HP216/216  
['はがねのつばさ', 'かみなりパンチ', 'たきのぼり', 'つのドリル']
選択=> つのドリル

相手 カイリュー HP216/216  
自分 バンギラス HP226/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> げんしのちから

update
|
|move|p1a: カイリュー|つのドリル|p2a: バンギラス
|split|p2
|-damage|p2a: バンギラス|0 fnt
|-damage|p2a: バンギラス|0 fnt
|-ohko
|faint|p2a: バンギラス
|
|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af9412+5f38dbeb63569b3772554f18, 5f37a6829949403bf6af940e+5f3835dad10c9203103a67e8

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in 


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|split|p2
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|turn|1
相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['おんがえし', 'かえんほうしゃ', 'ふぶき', 'サイコキネシス']
選択=> サイコキネシス

相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

update
|
|move|p2a: バンギラス|ばくれつパンチ|p1a: カビゴン
|-supereffective|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|151/292
|-damage|p1a: カビゴン|151/292
|-start|p1a: カビゴン|ねんりき
|-activate|p1a: カビゴン|ねんりき
|split|p1
|-damage|p1a: カビゴン|123/292|[from] ねんりき
|-damage|p1a: カビゴン|123/292|[from] ねんりき
|
|upkeep
|turn|2
相手 バンギラス HP226/226  
自分 カビゴン HP123/292  
['おんがえし', 'かえんほうしゃ', 'ふぶき', 'サイコキネシス']
選択=> ふぶき

相手 カビゴン HP123/292  
自分 バンギラス HP226/226  
['げんしのちから', '１０


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['ソーラービーム', 'かげぶんしん', 'なみのり', 'すてみタックル']
選択=> なみのり

update
|
|move|p1a: バンギラス|ばくれつパンチ|p2a: カビゴン|[miss]
|-miss|p1a: バンギラス
|move|p2a: カビゴン|なみのり|p1a: バンギラス
|-supereffective|p1a: バンギラス
|split|p1
|-damage|p1a: バンギラス|156/226
|-damage|p1a: バンギラス|156/226
|
|upkeep
|turn|2
相手 カビゴン HP292/292  
自分 バンギラス HP156/226  
['げんしのちから', '１０まんボルト', 'ばくれつパンチ', 'かえんほうしゃ']
選択=> ばくれつパンチ

相手 バンギラス HP156/226  
自分 カビゴン HP292/292  
['ソーラービーム', 'かげぶんしん', 'なみのり', 'すてみタックル']
選択=> なみのり

update
|
|move|p1a: バンギラス|ばくれつパ

In [70]:
display_target_battles("5f37a6829949403bf6af9410\\+5f3888187955353d68b9e44e")

DEBUG:__main__:match start: 5f37a6829949403bf6af9411+5f38b1286ec74d7bb2d8eb3e, 5f37a6829949403bf6af9410+5f3888187955353d68b9e44e

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|switch|p1a: バンギラス|バンギラス, L55, M|226/226
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['じしん', 'かいりき', 'げんしのちから', 'ばくれつパンチ']
選択=> ばくれつパンチ

相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> ばくれつパンチ

update
|
|move|p1a: バンギラス|ばくれつパンチ|p2a: カビゴン
|-supereffective|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|149/292
|-damage|p2a: カビゴン|149/292
|-start|p2a: カビゴン|ねんりき
|-activate|p2a: カビゴン|ねんりき
|split|p2
|-damage|p2a: カビゴン|121/292|[from] ねんりき
|-damage|p2a: カビゴン|121/292|[from] ねんりき
|
|upkeep
|turn|2
相手 カビゴン HP121/292  
自分


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ライコウ|ライコウ, L55|215/215
|switch|p1a: ライコウ|ライコウ, L55|215/215
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 ライコウ HP215/215  
['いあいぎり', 'どくどく', 'かみなり', 'ずつき']
選択=> かみなり

相手 ライコウ HP215/215  
自分 カビゴン HP292/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

update
|
|move|p1a: ライコウ|かみなり|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|206/292
|-damage|p2a: カビゴン|206/292
|move|p2a: カビゴン|のしかかり|p1a: ライコウ
|split|p1
|-damage|p1a: ライコウ|142/215
|-damage|p1a: ライコウ|142/215
|-status|p1a: ライコウ|par
|
|upkeep
|turn|2
相手 カビゴン HP206/292  
自分 ライコウ HP142/215 par 
['いあいぎり', 'どくどく', 'かみなり', 'ずつき']
選択=> かみなり

相手 ライコウ HP142/215 par 
自分 カビゴン HP206/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

update
|
|move|p2a: カビゴン|のしかかり|p1a: ライコ

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 カビゴン HP292/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

相手 カビゴン HP292/292  
自分 カビゴン HP292/292  
['ころがる', 'のしかかり', 'かみなりパンチ', 'バブルこうせん']
選択=> のしかかり

update
|
|move|p2a: カビゴン|のしかかり|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|209/292
|-damage|p1a: カビゴン|209/292
|move|p1a: カビゴン|のしかかり|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|214/292
|-damage|p2a: カビゴン|214/292
|-status|p2a: カビゴン|par
|
|upkeep
|turn|2
相手 カビゴン HP214/292 par 
自分 カビゴン HP209/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

相手 カビゴン HP209/292  
自分 カビゴン HP214/292 par 
['ころがる', 'のしかかり', 'かみなりパンチ', 'バブルこうせん']
選択=> のしかかり

update
|
|move|p1


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 カビゴン HP292/292  
['すなあらし', 'かみなりパンチ', 'のしかかり', 'なみのり']
選択=> のしかかり

相手 カビゴン HP292/292  
自分 カビゴン HP292/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

update
|
|move|p1a: カビゴン|のしかかり|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|213/292
|-damage|p2a: カビゴン|213/292
|move|p2a: カビゴン|のしかかり|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|213/292
|-damage|p1a: カビゴン|213/292
|
|upkeep
|turn|2
相手 カビゴン HP213/292  
自分 カビゴン HP213/292  
['すなあらし', 'かみなりパンチ', 'のしかかり', 'なみのり']
選択=> のしかかり

相手 カビゴン HP213/292  
自分 カビゴン HP213/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

update
|
|move|p2a: カビゴン|のしかかり|p1a: カビゴン
|split|p

|win|p1
DEBUG:__main__:match start: 5f37a6829949403bf6af940b+5f37a6dfdad3374a0e124a5b, 5f37a6829949403bf6af9410+5f3888187955353d68b9e44e

update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: ヤドキング|ヤドキング, L55, M|220/220
|switch|p1a: ヤドキング|ヤドキング, L55, M|220/220
|split|p2
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|switch|p2a: カビゴン|カビゴン, L55, M|292/292
|turn|1
相手 カビゴン HP292/292  
自分 ヤドキング HP220/220  
['サイコキネシス', 'じしん', 'とっしん', 'のしかかり']
選択=> のしかかり

相手 ヤドキング HP220/220  
自分 カビゴン HP292/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

update
|
|move|p2a: カビゴン|のしかかり|p1a: ヤドキング
|split|p1
|-damage|p1a: ヤドキング|146/220
|-damage|p1a: ヤドキング|146/220
|move|p1a: ヤドキング|のしかかり|p2a: カビゴン
|split|p2
|-damage|p2a: カビゴン|253/292
|-damage|p2a: カビゴン|253/292
|
|upkeep
|turn|2
相手 カビゴン HP253/292  
自分 ヤドキング HP146/220  
['サイコキネシス', 'じしん', 'とっしん', 'のしかかり']
選択=> のしかかり


update
|player|p1|p1||
|player|p2|p2||
|teamsize|p1|1
|teamsize|p2|1
|gametype|singles
|gen|2
|tier|[Gen 2] Custom Game
|rule|HP Percentage Mod: HP is shown in percentages
|
|start
|split|p1
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|switch|p1a: カビゴン|カビゴン, L55, M|292/292
|split|p2
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|switch|p2a: バンギラス|バンギラス, L55, M|226/226
|turn|1
相手 バンギラス HP226/226  
自分 カビゴン HP292/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> ばくれつパンチ

相手 カビゴン HP292/292  
自分 バンギラス HP226/226  
['じしん', 'ほのおのパンチ', 'どろかけ', 'げんしのちから']
選択=> げんしのちから

update
|
|move|p2a: バンギラス|げんしのちから|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|222/292
|-damage|p1a: カビゴン|222/292
|move|p1a: カビゴン|ばくれつパンチ|p2a: バンギラス
|-supereffective|p2a: バンギラス
|split|p2
|-damage|p2a: バンギラス|52/226
|-damage|p2a: バンギラス|52/226
|-start|p2a: バンギラス|ねんりき
|
|upkeep
|turn|2
相手 バンギラス HP52/226  
自分 カビゴン HP222/292  
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

相手 カビゴン HP222/292  
自分 バンギラス HP52/226  
['じしん', 'ほのおのパンチ', 'どろかけ', 'げんし

|turn|2
相手 ガルーラ HP159/231  
自分 カビゴン HP216/292 par 
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

相手 カビゴン HP216/292 par 
自分 ガルーラ HP159/231  
['ほのおのパンチ', 'ちきゅうなげ', 'どろかけ', 'のしかかり']
選択=> のしかかり

update
|
|move|p2a: ガルーラ|のしかかり|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|139/292 par
|-damage|p1a: カビゴン|139/292 par
|move|p1a: カビゴン|のしかかり|p2a: ガルーラ
|-crit|p2a: ガルーラ
|split|p2
|-damage|p2a: ガルーラ|32/231
|-damage|p2a: ガルーラ|32/231
|
|upkeep
|turn|3
相手 ガルーラ HP32/231  
自分 カビゴン HP139/292 par 
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

相手 カビゴン HP139/292 par 
自分 ガルーラ HP32/231  
['ほのおのパンチ', 'ちきゅうなげ', 'どろかけ', 'のしかかり']
選択=> のしかかり

update
|
|move|p2a: ガルーラ|のしかかり|p1a: カビゴン
|split|p1
|-damage|p1a: カビゴン|64/292 par
|-damage|p1a: カビゴン|64/292 par
|debug|move failed because it did nothing
|cant|p1a: カビゴン|par
|
|upkeep
|turn|4
相手 ガルーラ HP32/231  
自分 カビゴン HP64/292 par 
['れいとうパンチ', 'すなあらし', 'のしかかり', 'ばくれつパンチ']
選択=> のしかかり

相手 カビゴン HP64/292 par 
自分 ガルーラ HP32/231  
['ほのおのパンチ', 'ちきゅうなげ', 'どろかけ', 'のしかかり']
選択=> どろ

|win|p1
